In [116]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder

In [117]:
data = pd.read_csv("churn_Modelling.csv")
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [118]:
df = data.drop(['RowNumber','CustomerId','Surname'],axis=1)

In [119]:
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [120]:
le = LabelEncoder()
ohe = OneHotEncoder(handle_unknown='ignore')

In [121]:
df["Gender"] = le.fit_transform(df["Gender"])
df.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [122]:
geo_onehot_encoder = ohe.fit_transform(df[["Geography"]]).toarray()
geo_encoded_df = pd.DataFrame(geo_onehot_encoder, columns=ohe.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [123]:
df = pd.concat([df.drop('Geography',axis=1),geo_encoded_df],axis=1)

In [124]:
df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
9995,771,1,39,5,0.00,2,1,0,96270.64,0,1.0,0.0,0.0
9996,516,1,35,10,57369.61,1,1,1,101699.77,0,1.0,0.0,0.0
9997,709,0,36,7,0.00,1,0,1,42085.58,1,1.0,0.0,0.0
9998,772,1,42,3,75075.31,2,1,0,92888.52,1,0.0,1.0,0.0


In [125]:
X = df.drop('EstimatedSalary',axis=1)
y = df['EstimatedSalary']

In [126]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2,random_state=42)

In [127]:
X_train

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,Exited,Geography_France,Geography_Germany,Geography_Spain
9254,686,1,32,6,0.00,2,1,1,0,1.0,0.0,0.0
1561,632,1,42,4,119624.60,2,1,1,0,0.0,1.0,0.0
1670,559,1,24,3,114739.92,1,1,0,1,0.0,0.0,1.0
6087,561,0,27,9,135637.00,1,1,0,1,1.0,0.0,0.0
6669,517,1,56,9,142147.32,1,0,0,1,1.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
5734,768,1,54,8,69712.74,1,1,1,0,1.0,0.0,0.0
5191,682,0,58,1,0.00,1,1,1,0,1.0,0.0,0.0
5390,735,0,38,1,0.00,3,0,0,1,1.0,0.0,0.0
860,667,1,43,8,190227.46,1,1,0,1,1.0,0.0,0.0


In [128]:
# scaler three features

scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

In [129]:
import pickle

In [130]:
# Save the encoders and scaler for later use

with open('regression_gender.pkl','wb') as file:
    pickle.dump(le,file)

with open('regression_ohe.pkl','wb') as file:
    pickle.dump(ohe,file)

with open('regression_scaler.pkl','wb') as file:
    pickle.dump(scaler,file)

# ANN Regression Statement

In [131]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [132]:
# Build the model

model = Sequential([
    Dense(64, activation='relu',input_shape=(X_train.shape[1],)),
    Dense(32, activation='relu'),
    Dense(1) # Output layer
])

## compile the model

model.compile(optimizer='adam',loss='mean_absolute_error',metrics=['mae'])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 64)                832       
                                                                 
 dense_7 (Dense)             (None, 32)                2080      
                                                                 
 dense_8 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [133]:
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

# set up TensorBoard
log_dir = "regressionlogs/fit" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorboard_callback = TensorBoard(log_dir=log_dir, histogram_freq=1)

In [134]:
# Set up Early Stopping

early_stopping_callback = EarlyStopping(monitor='val_loss',patience=10,restore_best_weights=True)

In [135]:
# train the model

history = model.fit(
    X_train, y_train,
    validation_data = (X_test, y_test),
    epochs=100,
    callbacks = [early_stopping_callback, tensorboard_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 100386.1172 - mae: 100386.1172 - val_loss: 98546.3125 - val_mae: 98546.3125
Epoch 2/100
250/250 [==============================] - 0s 1ms/step - loss: 99738.4688 - mae: 99738.4688 - val_loss: 97233.0859 - val_mae: 97233.0859
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 97451.6953 - mae: 97451.6953 - val_loss: 93858.6719 - val_mae: 93858.6719
Epoch 4/100
250/250 [==============================] - 0s 1ms/step - loss: 92918.4922 - mae: 92918.4922 - val_loss: 88168.5078 - val_mae: 88168.5078
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 86164.8047 - mae: 86164.8047 - val_loss: 80587.4844 - val_mae: 80587.4844
Epoch 6/100
250/250 [==============================] - 0s 1ms/step - loss: 77895.7109 - mae: 77895.7109 - val_loss: 72227.5312 - val_mae: 72227.5312
Epoch 7/100
250/250 [==============================] - 0s 1ms/step - loss: 69418.2891 - mae: 69418.2891 

In [148]:
model.save('regression_model.h5')

d:\Conda\envs\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [136]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [145]:
%tensorboard --logdir regressionlogs/fit20250802-142930 

Reusing TensorBoard on port 6008 (pid 20444), started 0:00:02 ago. (Use '!kill 20444' to kill it.)